In [71]:
import numpy as np
import pandas as pd
import itertools

In [72]:
# reference code for generating 3d plots by index.
# # prepare some coordinates
# x, y, z = np.indices((8, 8, 1))

# # draw cuboids in the top left and bottom right corners, and a link between
# # them
# cube1 = (x < 3) & (y < 3) & (z < 1)
# cube2 = (x >= 5) & (y >= 5) & (z <1)
# #link = abs(x - y) + abs(y - z) + abs(z - x) <= 2

# # combine the objects into a single boolean array
# voxelarray = cube1 | cube2 #| link

# # set the colors of each object
# colors = np.empty(voxelarray.shape, dtype=object)
# #colors[link] = 'red'
# colors[cube1] = 'blue'
# colors[cube2] = 'green'

# # and plot everything
# ax = plt.figure().add_subplot(projection='3d')
# ax.voxels(voxelarray, facecolors=colors, edgecolor='k')

# plt.show()

In [73]:
#refer to cat pieces by colours. try not to forget em. if in doubt, ROY G BIV lightest hue to darkest

#lets exclude the third dimension to start with because that shit is hard and just encode these as the minimal size 
#binary matrix that can surround a given piece.
#generate containing array of 1, then remove gaps in rectangle
#sike, doing it as squares because it makes matrix transformations/uniqueness checks work better.

#let's start this all in two dimensions, which is probably gonna be a pain later, but should make this a lot easier.
#also, going to store these in a dictionary for indicing/iteration purposes

cat_states=dict()

white=np.full((2,3), "W", dtype=str)
white[0][1:] = ""
cat_states['white']=white

pink=np.full((3,3),"P", dtype=str)
pink[0][1:] = ""
pink[2][1]=""
cat_states['pink']=pink

red=np.full((3,4),"R", dtype=str)
red[0][1:] = ""
red[2][1:3]=""
cat_states['red']=red

yellow=np.full((3,3),"Y", dtype=str)
yellow[0][1:] = ""
cat_states['yellow']=yellow

mint=np.full((3,3), "M", dtype=str)
mint[0][0] = ""
mint[0][2] =""
mint[1][0] = ""
cat_states['mint']=mint

green=np.full((4,4),"G", dtype=str)
green[0][0:2] = ""
green[1][1:3] = ""
green[3][1:3] = ""
cat_states['green']=green

arctic=np.full((3,4), "A", dtype=str)
arctic[0][0:3]=""
cat_states['arctic']=arctic

sky=np.full((1,2), "S", dtype=str)
cat_states['sky']=sky

teal=np.full((2,2), "T", dtype=str)
teal[0][1]=""
cat_states['teal']=teal

indigo=np.full((2,2), "I", dtype=str)
cat_states['indigo']=indigo

violet=np.full((3,2), "V", dtype=str)
cat_states['violet']=violet

black=np.full((3,3), "B", dtype=str)
black[0][1]=""
cat_states['black']=black


In [74]:
#used for sorting cat placement order. heuristic of biggest to smallest by max height/width, then by area
cat_priority={"green":1,
              "arctic":2,
              "red":3,
              "black":4,
              "yellow":5,
              "mint" : 6,
              "pink": 7,
              "violet": 8,
              "white": 9,
              "indigo": 10,
              "teal": 11,
              "sky": 12}

In [75]:
#dumb manual function for removing duplicate arrays in a list of 2d numpy arrays (any m x n)

def _pad_to_square(array: np.array) -> np.array:
    m, n = array.shape
    #padding with "0" by default -- would be more consistent to use empty strings, but only planning to use
    #this within removing dup arays.
    if m == n:
        None
    elif m < n:
        vpad= n-m
        array = np.pad(array, [(0, vpad),(0,0)], "constant")
    elif m > n:
        hpad = m-n
        array = np.pad(array, [(0, 0),(0, hpad)], "constant")
    return array

#cleanup -- removes final column/row if it's '0' 
#lazy and hardcoding this to be only for final column/row which is true for this tileset
#padding will only ever add a max of 1 column or row
def _remove_padding(array: np.array) -> np.array:
    #if last column was padded to all '0:
    if (array[:, -1] == '0').all():
        #remove last column
        array = array[:, 0:-1]
    #if last row padded to '0
    elif (array[-1, :] == '0').all():
        array = array[0:-1, :] 
    return array

def _remove_dup_arrays(array_list: list[np.array]) -> list[np.array]:
    #pad all arrays to squares
    padded_arrays=[_pad_to_square(array) for array in array_list]
    #get unique arrays
    unique_padded_arrays = list(np.unique([array for array in padded_arrays], axis=0))
    #unpad 
    unique_arrays=[_remove_padding(array) for array in unique_padded_arrays]
    
    return unique_arrays

In [76]:
def _generate_rotations(cat: np.array)-> np.array:
    """Generates four rotation states of cat piece from initial state.

    Args:
        cat (np.array): Numpy array representing cat shape

    Returns:
        list[np.array]: Rotation states of of shape
    """
    is_square = (cat.shape[0] == cat.shape[1])
    states=[cat]
    for i in range(3):
        #rotate clockwise and add to state list
        cat=np.rot90(cat)
        states.append(cat)
    #remove any duplicates.
    # if is_square:
    #     states=list(np.unique(states, axis=0))
    # else:
    #     #np.unique needs arrays to be of same dimension, pad to square.
    #     states=_remove_dup_arrays(states)
    return states

def _generate_states(cat: np.array)-> np.array:
    """Generates up to 8 rotation states of cats -- rotations, flip then rotations, then removes any duplicates.

    Args:
        cat (np.array): Numpy array representing cat shape.

    Returns:
        np.array: All orientations of shape induced by rotation and flip
    """
    
    #get rotation states
    states=_generate_rotations(cat)
    #get flip states
    f_states= _generate_rotations(np.fliplr(cat))

    states.extend(f_states)
    states = _remove_dup_arrays(states)
    return states


In [77]:
#dictionary of all shape states

for key, val in cat_states.items():
    cat_states[key]= _generate_states(val)

In [96]:
#missing spaces will be set to value "X"
#blocked indices is list of (m,n) tuples where an X is placed at the index m,n
def init_puzzle_grid(max_height, max_width, blocked_indices=None):
    grid= np.full((max_height, max_width), "", dtype=str)

    if (blocked_indices != None):
        for index in  blocked_indices:
            m, n = index
            grid[m,n] = "X"

    return grid

In [95]:
init_puzzle_grid(8, 7,blocked_indices=((2,1),
                                       (2, 5),
                                       (5, 1),
                                       (5, 5)))

array([['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', 'X', '', '', '', 'X', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', ''],
       ['', 'X', '', '', '', 'X', ''],
       ['', '', '', '', '', '', ''],
       ['', '', '', '', '', '', '']], dtype='<U1')

In [79]:
def _get_next_cat(cats: list, priorities=cat_priority):
    prior_list= [priorities[cat] for cat in cats]
    return prior_list
def _check_solved(state: np.array) -> bool:
    #return false if any empty strings in puzzle state
    return np.all(state != "")
def _is_valid_grid(grid):
    #just checks if all strings in grid are 1 character or less. if anything more, means we've overlapped somewhere.
    return (np.strings.str_len(grid) <= 1).all()
        

In [80]:
#using these to hold state for a given orientation as we iterate thru possible placements for it in a puzzle
#this is probably dumb
#this is definitely ugly.
#not sure cat_states needs to be passed through all of this but losing track of what scope i need to call it in.
class Cat_Placement:
    def __init__(self, colour, grid_init, i = -1, j= -1, orientation_idx=0, cat_states=cat_states):
        self.colour = colour
        self.orientation_idx = orientation_idx
        self.orientations = cat_states[colour]
        self.orientation = self.orientations[orientation_idx]
        self.cat_size_m= self.orientation.shape[0] #rows
        self.cat_size_n=self.orientation.shape[1] #cols
        self.i=i
        self.j=j
        self.make_cat_placement_index_gen(grid_init)

    def make_cat_placement_index_gen(self, grid_init):
        grid_height = grid_init.shape[0]
        grid_width = grid_init.shape[1]
        i_max=grid_height-self.cat_size_m
        j_max=grid_width - self.cat_size_n
        o_max = len(self.orientations) - 1
        
        def idx_gen_func(self, i_max=i_max, j_max=j_max, o_max=o_max):
            i, j, o = 0,0,0 #col, row, orientation index
            while True:
                self.i = i
                self.j= j
                self.orientation_idx = o
                yield i, j, o
                #increment column position
                #print("incrementing column index")
                j += 1
                if j >j_max:
                    #if at max columns, return to 0 indexed column and move one row down
                    #print("exceeded column count")
                    j=0
                    i+=1
                if i > i_max:
                    # if at max rows, return to 0,0 and try next orientation
                   # print("exceeded rowcount")
                    i = 0
                    j= 0 
                    #print("updating orientation index")
                    o +=1
                    #update orientation
                    #print("updating self orientation")
                    self.orientation = self.orientations[o]
                    self.cat_size_m= self.orientation.shape[0] #rows
                    self.cat_size_n=self.orientation.shape[1] #cols
                   # print("self orientation update succeded.")
                if o > o_max:
                    raise Exception("All indices exhausted.")
        self.idx_gen = idx_gen_func(self)

In [104]:
#unsure if this will actually speed things up.
#custom comparison that immediately throws things out if the index to place things at is has an object in it
# and orientation[0,0] is not empty.

def _check_idx_blocked(grid_state, o_plc):
    o_idx= o_plc.orientation[0,0]
    i = o_plc.i
    j = o_plc.j
    g_idx = grid_state[i,j]

    #if both grid and orientation indices are non-empty
    if (g_idx != "") and (o_idx != ""):
        return True
    else:
        #else this might work, return true.
        return False

In [111]:
def _place_o_at_idx(grid_state, o_plc):
    #make placement array
    #returns results of placement regardless if valid or not
    i = o_plc.i
    j = o_plc.j
    cat_size_m= o_plc.cat_size_m
    cat_size_n= o_plc.cat_size_n
    orientation=o_plc.orientation
    placement = grid_state[i: i + cat_size_m, j: j+ cat_size_n] + orientation
    return placement


def place_orientation(grid_state, o_plc):
    #trigger next placement index and try to place.w
    #loop terminates when it finds something to place
    #otherwise throws exception (in index generator, this might be dumb, i don't know shit about exception handling)
    #upon running out of indices to "next" to.

    #AT SOME POINT, NEEDS REAL EXCEPTION HANDLING SO THAT IF THIS ERRORS, WE REVERT PLACEMENT OF PREVIOUS PIECE.
    while True:
        #print("trying next index")
        next(o_plc.idx_gen)
        #print("next index sucessfully retrieved")
        #print("trying to place")

        if _check_idx_blocked(grid_state, o_plc):
            continue
        
        placement= _place_o_at_idx(grid_state, o_plc)
        #print("placed without error")
        #print(placement)
        if _is_valid_grid(placement):
            #if so, update grid state and return it
            #print("placement was valid")
            #print(placement)
            grid_state[o_plc.i: o_plc.i + o_plc.cat_size_m, o_plc.j: o_plc.j+ o_plc.cat_size_n] = placement
            break
    return grid_state


In [82]:
def place_cat(grid_state, cat_col, cat_states=cat_states, o_plc=None):
    #for some grid state and some cat, add cat to first possible open state (if possible)
    
    #if o_plc=None, creates cat_placement obhject from grid state, colour, and states
    #otherwise, retrieves placement object passed to it.
    #probably should check that o_plc is actually a cat object
    if o_plc == None:
        #print("initiating cat object")
        o_plc=Cat_Placement(cat_col, grid_state, cat_states=cat_states)
        #print("cat object initated")
    try:
        #try to place o_plc at it's current o,i,j index
        grid_state=place_orientation(grid_state, o_plc)
    except:
        raise Exception(f"Couldn't place {cat_col} token")

    return grid_state, o_plc

In [83]:
def solve_puzzle(cats: list[str], grid_init: np.array, symmetry=True):
    #sort cats by priority
    #doing this in reverse order so that we can use .pop to get the highest priority piece.
    cats.sort(key = lambda x: -cat_priority[x])
    #initiate dictionary of cat placement objects labled by colours. this is kinda ugly.
    cat_placements={}
    for col in cats:
        cat_placements[col] = None

    to_place= cats.copy()
    placed_cols=[]
    grid_state=grid_init.copy()

    #place first cat. special case:
    #if symmetry=True, (m x n) puzzle grid has horizontal/vertical symmetry
    #this means while searching solutions we can hold the inital cat to be placed within
    #the (ceiling(m/2), ceiling(n/2)) top left subgrid.

    #next case: place it wherever and brute force. real high level stuff.
    while len(to_place) > 0:
        #get highest priority piece
        cat_col=to_place[-1]
        #print(f"Placing {cat_col}")

        placement=cat_placements[cat_col]
        try:
            #try to place cat
            grid_state, placed_cat=place_cat(grid_state, cat_col, o_plc=placement)
        except:
            #all possible placements of next_cat exhausted without success
            #revert previous piece, place in next possible spot, and try again.
            #print(f"Placement failed")
            prev_cat = placed_cols.pop()
            to_place.append(prev_cat)
            #revert placement of current cat in dict
            cat_placements[cat_col]=None
     
            #REMOVE PREVIOUS PIECE FROM GRID STATE.
            #just replace matrix character lbl with empty string.
            cat_matrix_lbl=prev_cat.upper()[0]
            #print(f"Removing {prev_cat} and placing again")
            grid_state = np.strings.replace(grid_state, cat_matrix_lbl, "")
            
        else:
            #cat successfully placed
            #print(f"{cat_col} placed")
            #remove cat label from to_place and add to placed_cols
            placed_cols.append(to_place.pop())
            #log placement object to cat_placements
            cat_placements[cat_col]=placed_cat
            #set flag for init (for next piece)


        #print(grid_state)
      
    print(grid_state)
    return grid_state, cat_placements


In [86]:
grid1=init_puzzle_grid(5,5)
grid, cats=solve_puzzle(['black', 'mint', 'violet', 'sky', 'teal'], grid1)

[['S' 'M' 'M' 'T' 'T']
 ['S' 'M' 'M' 'M' 'T']
 ['B' 'M' 'B' 'V' 'V']
 ['B' 'B' 'B' 'V' 'V']
 ['B' 'B' 'B' 'V' 'V']]


In [87]:
grid5=init_puzzle_grid(6,7)
grid, cats=solve_puzzle(['arctic','violet', 'sky', 'teal', 'yellow', 'black', 'red'], grid5)

[['V' 'V' 'R' 'R' 'S' 'S' 'A']
 ['V' 'V' 'R' 'A' 'A' 'A' 'A']
 ['V' 'V' 'R' 'A' 'A' 'A' 'A']
 ['Y' 'R' 'R' 'R' 'B' 'B' 'B']
 ['Y' 'Y' 'Y' 'T' 'T' 'B' 'B']
 ['Y' 'Y' 'Y' 'T' 'B' 'B' 'B']]


In [112]:
grid14=init_puzzle_grid(7,6)
grid, cats=solve_puzzle(['pink','arctic', 'violet', 'green', 'mint', 'sky', 'teal'], grid14)

[['T' 'T' 'G' 'S' 'S' 'G']
 ['T' 'M' 'G' 'G' 'G' 'G']
 ['M' 'M' 'G' 'P' 'P' 'G']
 ['M' 'M' 'M' 'P' 'G' 'G']
 ['V' 'V' 'P' 'P' 'P' 'A']
 ['V' 'V' 'A' 'A' 'A' 'A']
 ['V' 'V' 'A' 'A' 'A' 'A']]


In [110]:
grid16=init_puzzle_grid(8, 7, blocked_indices=((2,1),
                                       (2, 5),
                                       (5, 1),
                                       (5, 5)))

grid, cats = solve_puzzle(['black',
                           'indigo',
                           'arctic',
                           'yellow',
                           'violet',
                           'mint',
                           'teal',
                           'sky',
                           'red'], grid16)

[['V' 'V' 'V' 'T' 'T' 'M' 'M']
 ['V' 'V' 'V' 'T' 'M' 'M' 'M']
 ['B' 'X' 'B' 'R' 'R' 'X' 'M']
 ['B' 'B' 'B' 'R' 'I' 'I' 'S']
 ['B' 'B' 'B' 'R' 'I' 'I' 'S']
 ['Y' 'X' 'R' 'R' 'R' 'X' 'A']
 ['Y' 'Y' 'Y' 'A' 'A' 'A' 'A']
 ['Y' 'Y' 'Y' 'A' 'A' 'A' 'A']]
